# Import Dataset

In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_olivetti_faces
from sklearn.preprocessing import LabelBinarizer

data = fetch_olivetti_faces(shuffle = True)
labels = LabelBinarizer().fit_transform(data.target)

# Training Data
images_DataTraining = data.data[0:299]
labels_DataTraining = labels[0:299]

# Validation Data
images_DataValidation = data.data[299:349]
labels_DataValidation = labels[299:349]

# Testing Data
images_DataTesting = data.data[349:399]
labels_DataTesting = labels[349:399]

# Convolution and Pooling Layers

Input Layer -> 1st Convolutional Layer -> 1st Pooling Layer -> 2nd Convolutional Layer -> 2nd Pooling Layer -> 3rd Convolutional Layer -> 3rd Pooling Layer -> Fully Connected Layer -> Output Layer

In [5]:
x = tf.placeholder(tf.float32, [None, 4096])
y_ = tf.placeholder(tf.float32, [None, 40])

# Add function to create weight variable 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

# Add function to create bias variable 
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# Add convolution function
def conv2d(x, W):
    return(tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = "SAME"))

# Add Pooling function
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

print(x.shape)

# Reshape the data for inputting it in the input layer
x_image = tf.reshape(x, [-1, 64, 64, 1])

print(x_image.shape)

# Create 1st Convolutional and Pooling Layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


# Create 2nd Convolutional and Pooling Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


# Create 3rd Convolutional and Pooling Layer
W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

# 1st Fully Connected Layer
W_fc1 = weight_variable([128*8*8, 1024])
b_fc1 = bias_variable([1024])

# Reshape the last convolutional layer output to connect with Fully Connected Layer
h_pool2_flat = tf.reshape(h_pool3, [-1, 128*8*8])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Add the dropout layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output Layer
W_fc2 = weight_variable([1024, 40])
b_fc2 = bias_variable([40])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

(?, 4096)
(?, 64, 64, 1)


# Training

In [3]:
# Network base of Computation
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y_conv))
training_op  = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Prediction Measures
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Mention about the number of epochs training has to happen
#epochs = 1100

saver = tf.train. Saver()

n_epochs = 100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict = {x : images_DataTraining, y_:labels_DataTraining, keep_prob : 0.5})
        acc_train = accuracy.eval(feed_dict={x : images_DataTraining, y_:labels_DataTraining, keep_prob : 1.0})
        acc_validation = accuracy.eval(feed_dict={x : images_DataValidation, y_:labels_DataValidation, keep_prob : 1.0})
        print(epoch, "Train accuracy", acc_train, "Validation accuracy", acc_validation)
    save_path = saver.save(sess, "./my_model_find.ckpt")    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

0 Train accuracy 0.050167225 Validation accuracy 0.02
1 Train accuracy 0.06688963 Validation accuracy 0.06
2 Train accuracy 0.053511705 Validation accuracy 0.06
3 Train accuracy 0.06354515 Validation accuracy 0.06
4 Train accuracy 0.05685619 Validation accuracy 0.0
5 Train accuracy 0.06354515 Validation accuracy 0.02
6 Train accuracy 0.06688963 Validation accuracy 0.02
7 Train accuracy 0.07692308 Validation accuracy 0.02
8 Train accuracy 0.07023411 Validation accuracy 0.04
9 Train accuracy 0.06020067 Validation accuracy 0.02
10 Train accuracy 0.07023411 Validation accuracy 0.02
11 Train accuracy 0.06354515 Validation accuracy 0.02
12 Train accuracy 0.09030101 Validation accuracy 0.04
13 Train accuracy 0.10702341 Validation accuracy 0.08
14 Train accuracy 0.10367893 Validation accuracy 0.04
15 Trai

In [4]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_find.ckpt")
    acc_testing = accuracy.eval(feed_dict={x : images_DataTesting, y_:labels_DataTesting, keep_prob : 1})
    print("Testing accuracy", acc_testing)    

INFO:tensorflow:Restoring parameters from ./my_model_find.ckpt
Testing accuracy 0.8
